## ĐẠI HỌC KHOA HỌC TỰ NHIÊN - KHOA CÔNG NGHỆ THÔNG TIN 

## Trực quan hoá dữ liệu

### Lab02: Trực quan hoá dữ liệu với Tableau

Nhóm 5
1. 1712117 – Nguyễn Huỳnh Thảo Nhi
2. 1712710 – Lê Quang Quí
3. 1712713 – Lê Bá Quyền
4. 1712775 – Nguyễn Lê Trường Thành


### Import  thư viện

In [14]:
import requests
import pandas as pd   
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from bs4 import BeautifulSoup
import lxml.html as lh
import seaborn as sns
import numpy as np
from sklearn import datasets

### Crawl Dữ liệu thống kê từng ngày ca nhiễm virus Covid-19 từ tổ chức Worldometer

In [2]:
URL = 'https://www.worldometers.info/coronavirus/' #the website the data is extracted
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

In [3]:
table = soup.find(id='nav-tabContent')
table = table.find(id = 'nav-today')
table = table.find(id = '')
table = table.find(id = 'main_table_countries_today')

In [4]:
table_rows = table.find_all('tr') #finds all the tables with the tag tr in html
l = [] 
for tr in table_rows: 
    row = []
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    if (len(row) == 0 or '\n' in row[1] ):
        continue  
    row_temp = row[:7] + row[8:10]
    l.append(row_temp) 

dataset = pd.DataFrame(l, columns=["Ranking","Country","Total Cases","New Cases","Total Deaths","New Deaths","Total Recoverd","Active Cases","Serious Cases"])
dataset

,Ranking,Country,Total Cases,New Cases,Total Deaths,New Deaths,Total Recoverd,Active Cases,Serious Cases
0,,World,"10,280,397","+42,110","505,145","+1,067","5,580,959","4,194,293","57,483"
1,1,USA,"2,637,439",+362,"128,443",+6,"1,093,612","1,415,384","15,825"
2,2,Brazil,"1,345,470",+216,"57,659",+1,"733,848","553,963","8,318"
3,3,Russia,"641,156","+6,719","9,166",+93,"403,430","228,560","2,300"
4,4,India,"550,395","+1,198","16,506",+19,"322,699","211,190","8,944"
...,...,...,...,...,...,...,...,...,...
219,,Total:,"2,235,585","+24,152","55,296",+510,"1,460,197","720,092","19,122"
220,,Total:,"386,357","+1,138","9,719",+21,"185,747","190,891",931
221,,Total:,"9,407",+83,126,,"8,599",682,1
222,,Total:,721,,15,,651,55,4


### Lưu dữ liệu hàng ngày vào file .csv 

In [5]:
import datetime
datestr = datetime.date.today().strftime("%Y%m%d")
dataset.to_csv('CoronaData_{}.csv'.format(datestr))
dataset

,Ranking,Country,Total Cases,New Cases,Total Deaths,New Deaths,Total Recoverd,Active Cases,Serious Cases
0,,World,"10,280,397","+42,110","505,145","+1,067","5,580,959","4,194,293","57,483"
1,1,USA,"2,637,439",+362,"128,443",+6,"1,093,612","1,415,384","15,825"
2,2,Brazil,"1,345,470",+216,"57,659",+1,"733,848","553,963","8,318"
3,3,Russia,"641,156","+6,719","9,166",+93,"403,430","228,560","2,300"
4,4,India,"550,395","+1,198","16,506",+19,"322,699","211,190","8,944"
...,...,...,...,...,...,...,...,...,...
219,,Total:,"2,235,585","+24,152","55,296",+510,"1,460,197","720,092","19,122"
220,,Total:,"386,357","+1,138","9,719",+21,"185,747","190,891",931
221,,Total:,"9,407",+83,126,,"8,599",682,1
222,,Total:,721,,15,,651,55,4


### Tiền xử lý dữ liệu

In [6]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224 entries, 0 to 223
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Ranking         224 non-null    object
 1   Country         224 non-null    object
 2   Total Cases     224 non-null    object
 3   New Cases       224 non-null    object
 4   Total Deaths    224 non-null    object
 5   New Deaths      224 non-null    object
 6   Total Recoverd  224 non-null    object
 7   Active Cases    224 non-null    object
 8   Serious Cases   224 non-null    object
dtypes: object(9)
memory usage: 15.9+ KB


In [7]:
# Function to Clean the DataSet
def dataframeCleaner(dataset):
  
    for columnname in dataset: #looping through titles of the table 
        temp = []     
        for column in dataset[columnname]:   #geting column elements for the each title
            column = str(column)
            column = column.replace(',','')# Removing unwanted data clutter
            column = column.replace('+','')#Removing unwanted '+'sign  \
            try:   
                column = int(column)
            except:
                pass
            
            temp.append(column)
        dataset[columnname] = temp
        
 #   dataset = dataset.drop(dataset.tail(5).index)          # Deleting the last row   
    dataset = dataset.replace('N/A', '', regex=True)
    dataset = dataset.replace(r'^\s*$', 0, regex=True)     # Converting empty string to 0
    dataset.replace(['\n'], '', regex=True, inplace=True)
    dataset.replace([','], '', regex=True, inplace=True)
    return dataset

### Dữ liệu sau khi được xử lý

In [8]:
dataset = dataframeCleaner(dataset) 
dataset[dataset["Country"] == "Asia"]
dataset

,Ranking,Country,Total Cases,New Cases,Total Deaths,New Deaths,Total Recoverd,Active Cases,Serious Cases
0,0,World,10280397,42110,505145,1067,5580959,4194293,57483
1,1,USA,2637439,362,128443,6,1093612,1415384,15825
2,2,Brazil,1345470,216,57659,1,733848,553963,8318
3,3,Russia,641156,6719,9166,93,403430,228560,2300
4,4,India,550395,1198,16506,19,322699,211190,8944
...,...,...,...,...,...,...,...,...,...
219,0,Total:,2235585,24152,55296,510,1460197,720092,19122
220,0,Total:,386357,1138,9719,21,185747,190891,931
221,0,Total:,9407,83,126,0,8599,682,1
222,0,Total:,721,0,15,0,651,55,4


In [9]:
datestr = datetime.date.today().strftime("%Y%m%d")
dataset.to_csv('CoronaData_{}_HorseColic.csv'.format(datestr))
dataset

,Ranking,Country,Total Cases,New Cases,Total Deaths,New Deaths,Total Recoverd,Active Cases,Serious Cases
0,0,World,10280397,42110,505145,1067,5580959,4194293,57483
1,1,USA,2637439,362,128443,6,1093612,1415384,15825
2,2,Brazil,1345470,216,57659,1,733848,553963,8318
3,3,Russia,641156,6719,9166,93,403430,228560,2300
4,4,India,550395,1198,16506,19,322699,211190,8944
...,...,...,...,...,...,...,...,...,...
219,0,Total:,2235585,24152,55296,510,1460197,720092,19122
220,0,Total:,386357,1138,9719,21,185747,190891,931
221,0,Total:,9407,83,126,0,8599,682,1
222,0,Total:,721,0,15,0,651,55,4


### Lấy dữ liệu đã lưu về và xử lý vào ngày 10/06/2020

In [10]:
dataset = pd.read_csv('CoronaData_20200610_HorseColic.csv')
dataset

,Unnamed: 0,Ranking,Country,Total Cases,New Cases,Total Deaths,New Deaths,Total Recoverd,Active Cases,Serious Cases
0,0,0,World,7387471,75947,415779,2807,3640755,3330937,53935
1,1,1,USA,2053217,7668,114408,260,789558,1149251,16875
2,2,2,Brazil,747561,5477,38701,204,325602,383258,8318
3,3,3,Russia,493657,8404,6358,216,252783,234516,2300
4,4,4,UK,290143,1003,41128,245,0,0,516
...,...,...,...,...,...,...,...,...,...,...
219,219,0,Total:,1459822,35552,37041,744,885327,537454,15092
220,220,0,Total:,207089,1569,5589,21,93711,107789,499
221,221,0,Total:,8887,9,124,0,8332,431,3
222,222,0,Total:,721,0,15,0,651,55,4


In [11]:
FEATURE_NAMES = ['Total Deaths', 'New Deaths','Total Recoverd','Active Cases','Serious Cases']

In [35]:
from pandas import DataFrame
import statsmodels.api as sm

df = DataFrame(dataset,columns=['New Cases','Total Deaths','New Deaths','Total Recoverd','Active Cases','Serious Cases']) 
X = df[['New Cases','Active Cases']]
Y = df['Serious Cases']
X = sm.add_constant(X)
model = sm.OLS(Y, X).fit()
predictions = model.predict(X) 

print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:          Serious Cases   R-squared:                       0.986
Model:                            OLS   Adj. R-squared:                  0.985
Method:                 Least Squares   F-statistic:                     7543.
Date:                Mon, 29 Jun 2020   Prob (F-statistic):          4.21e-204
Time:                        22:04:31   Log-Likelihood:                -1773.9
No. Observations:                 224   AIC:                             3554.
Df Residuals:                     221   BIC:                             3564.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           10.2060     45.440      0.225   

In [47]:

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df = DataFrame(dataset,columns=['New Cases','Total Deaths','New Deaths','Total Recoverd','Active Cases','Serious Cases']) 
scaler.fit(df)
scaled_data = scaler.transform(df)

# we will specify number of components as 2
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca.fit(scaled_data)

# Now we can transform this data to its first 2 principal components.
x_pca = pca.transform(scaled_data)

print(scaled_data.shape)
print(x_pca.shape)

# We’ve reduced 30 dimensions to just 2! Let’s plot these two dimensions out!

(224, 6)
(224, 2)


In [61]:
import seaborn as seabornInstance 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
%matplotlib inline
X = dataset['Active Cases'].values.reshape(-1,1)
y = dataset['Serious Cases'].values.reshape(-1,1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
regressor = LinearRegression()  
regressor.fit(X_train, y_train) #training the algorithm

#To retrieve the intercept:
print(regressor.intercept_)
#For retrieving the slope:
print(regressor.coef_)

y_pred = regressor.predict(X_test)
df = pd.DataFrame({'Actual': y_test.flatten(), 'Predicted': y_pred.flatten()})
df

[56.04484045]
[[0.01608476]]


,Actual,Predicted
0,0,59.245708
1,0,56.044840
2,2,58.071520
3,0,63.508169
4,0,56.044840
5,0,60.805929
6,90,156.880201
7,0,56.044840
8,5,75.443061
9,1077,1746.054493
